In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense,Input,Dropout,GlobalAveragePooling2D,Flatten,Conv2D,BatchNormalization,Activation,MaxPooling2D
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.optimizers import Adam,SGD,RMSprop

In [14]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Define paths
train_folder_path = r"C:\Emotion detection deep learning\images\train"

# Emotion categories (assume you have folders named as emotions)
emotions = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

# Load and preprocess images
def load_images(folder_path):
    images = []
    labels = []
    for emotion in emotions:
        emotion_folder = os.path.join(folder_path, emotion)
        if not os.path.exists(emotion_folder):
            print(f"Folder not found: {emotion_folder}")
            continue
        for img_name in os.listdir(emotion_folder):
            img_path = os.path.join(emotion_folder, img_name)
            if not img_name.endswith(('jpg', 'jpeg', 'png', 'gif')):
                continue
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE) # Read in grayscale
            img = cv2.resize(img, (48, 48)) # Resize to 48x48 pixels
            images.append(img)
            labels.append(emotion)
    return np.array(images), np.array(labels)

# Load images and labels
images, labels = load_images(train_folder_path)

if len(images) == 0:
    raise ValueError("No images found in the specified folders. Please check the paths and folder names.")

# Normalize pixel values
images = images / 255.0

# Encode labels
label_to_index = {emotion: index for index, emotion in enumerate(emotions)}
labels = np.array([label_to_index[label] for label in labels])
labels = to_categorical(labels, num_classes=len(emotions))

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)
X_train = X_train.reshape(-1, 48, 48, 1)
X_val = X_val.reshape(-1, 48, 48, 1)

   



In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def build_model():
    model = Sequential()

    # First convolutional layer
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # Second convolutional layer
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # Third convolutional layer
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # Flatten and dense layers
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(len(emotions), activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

model = build_model()

    

c:\Users\prabh\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [24]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('emotion_detection_model.keras', save_best_only=True)

from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Fit the model using augmented data
datagen.fit(X_train)

history = model.fit(
    datagen.flow(X_train, y_train, batch_size=64),
    validation_data=(X_val, y_val),
    epochs=50,
    callbacks=[early_stopping, model_checkpoint]
)



Epoch 1/50


c:\Users\prabh\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


361/361 ━━━━━━━━━━━━━━━━━━━━ 58s 154ms/step - accuracy: 0.3519 - loss: 1.6704 - val_accuracy: 0.5336 - val_loss: 1.2306
Epoch 2/50
361/361 ━━━━━━━━━━━━━━━━━━━━ 59s 162ms/step - accuracy: 0.3764 - loss: 1.5935 - val_accuracy: 0.5398 - val_loss: 1.2177
Epoch 3/50
361/361 ━━━━━━━━━━━━━━━━━━━━ 81s 223ms/step - accuracy: 0.4022 - loss: 1.5340 - val_accuracy: 0.5466 - val_loss: 1.2039
Epoch 4/50
361/361 ━━━━━━━━━━━━━━━━━━━━ 52s 142ms/step - accuracy: 0.4108 - loss: 1.5207 - val_accuracy: 0.5410 - val_loss: 1.2033
Epoch 5/50
361/361 ━━━━━━━━━━━━━━━━━━━━ 32s 88ms/step - accuracy: 0.4077 - loss: 1.5103 - val_accuracy: 0.5329 - val_loss: 1.2105
Epoch 6/50
361/361 ━━━━━━━━━━━━━━━━━━━━ 62s 173ms/step - accuracy: 0.4104 - loss: 1.5033 - val_accuracy: 0.5356 - val_loss: 1.2072
Epoch 7/50
361/361 ━━━━━━━━━━━━━━━━━━━━ 31s 86ms/step - accuracy: 0.4133 - loss: 1.5102 - val_accuracy: 0.5330 - val_loss: 1.2176
Epoch 8/50
361/361 ━━━━━━━━━━━━━━━━━━━━ 21s 58ms/step - accuracy: 0.4222 - loss: 1.4930 - val_ac

In [25]:
# Evaluate on validation data
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')


181/181 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.5720 - loss: 1.1358
Validation Loss: 1.1321589946746826
Validation Accuracy: 0.5691240429878235


In [26]:
import numpy as np
import cv2
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('emotion_detection_model.keras')

# Function to predict emotion from an image
def predict_emotion(img_path):
    # Load and preprocess the image
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (48, 48))
    img = img / 255.0
    img = np.reshape(img, (1, 48, 48, 1))
    
    # Predict the emotion
    predictions = model.predict(img)
    max_index = np.argmax(predictions[0])
    
    # Emotion categories
    emotions = ['anger', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
    
    # Return the predicted emotion
    return emotions[max_index]

# Example usage
img_path = r"C:\Users\prabh\Downloads\WhatsApp Image 2024-12-12 at 12.28.25 AM.jpeg"
predicted_emotion = predict_emotion(img_path)
print(f"The predicted emotion is: {predicted_emotion}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
The predicted emotion is: happy
